<a href="https://colab.research.google.com/github/bhardwaj1230/Natural-Language-Processing/blob/master/KL_divergence_and_Domain_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import re
import os
import pickle
import random
import numpy as np
from tqdm import tqdm


In [0]:
# Get Domain of all data :

def read_data(loc):
    objects = []
    with (open(loc, "rb")) as openfile: 
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    return objects

address_not_flagged = read_data("/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/complete_not_flagged/not_flagged_loc.en-fr")
address_flagged = read_data("/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/complete_flagged/all_loc.en-fr")

address_not_flagged = [item for ad in address_not_flagged for item in ad]
address_flagged = [item for ad in address_flagged for item in ad]

address = address_not_flagged + address_flagged

address = set(address)
locations=[]
for item in address:
    locations.append(item[:-2])

locations = list(set(locations))

new_location = []
for loc in locations:
    new_location.append(loc.replace('/space/project/portage/corpora/BtB/MegaCorpus_2019-02-14/', '/gpfs/fs2c/nrc/ict/nrc_collab_aitq/'))


def get_data(loc):

    cnt_e = 0
    counts = {}

    for en_fr in tqdm(loc):
        if os.path.exists(en_fr+str('fr')) and os.path.exists(en_fr+str('en')):

            with open(en_fr+str('fr')) as f, open(en_fr+str('en')) as e:
                domain_folder = en_fr.replace('/gpfs/fs2c/nrc/ict/nrc_collab_aitq/corpus_renamed/','')
                for fr, en in zip(f, e):
                    cnt_e +=1
                if domain_folder.split('/')[1] in counts:
                    counts[domain_folder.split('/')[1]] += cnt_e
                else:
                    counts[domain_folder.split('/')[1]] = cnt_e
                cnt_e = 0
    
    with open('/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/BT_Doamin_distribution.en-fr','wb') as w:
        pickle.dump(counts, w)

get_data(new_location)

In [0]:
# Get domain from downloaded 14M data for NMT :


with open('/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/train/flag_not_flagged/train.en-fr.en','r') as r:
	flag_en = [item for item in r.readlines()]

with open('/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/train/flag_not_flagged/train.en-fr.fr','r') as r:
	flag_fr = [item for item in r.readlines()]

def join(a,b):
    en_fr = []
    for x,y in tqdm(zip(a,b)):
        x = str(x)
        y = str(y)
        en_fr.append(' @$$$$@ '.join([x, y]))
    return en_fr

flagged = join(flag_fr , flag_en)    
flagged = set(flagged)
flag_en = 0
flag_fr = 0 

def get_data(loc):
    
    data_info = []
    
    for en_fr in tqdm(loc):
        if os.path.exists(en_fr+str('fr')) and os.path.exists(en_fr+str('en')):

            with open(en_fr+str('fr')) as f, open(en_fr+str('en')) as e:
                for idx, (line_f, line_e) in enumerate(zip(f,e)):

                    if line_f+' @$$$$@ '+line_e in flagged:
                        #print((line_f.strip('\n'),line_e.strip('\n'),en_fr,idx))
                        data_info.append((line_f.strip('\n'),line_e.strip('\n'),en_fr,idx))

    with open('/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/data_info.en-fr','wb') as w:
        pickle.dump(data_info, w)

    
get_data(new_location)


In [0]:

with open('/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/domain_distribution/14M_BT_domain_info.en-fr', 'rb') as handle:
    bt_14 = pickle.load(handle)

data_14M =[]
for f,e,loc,idx in bt_14: 
    loc_clean = loc.replace('/gpfs/fs2c/nrc/ict/nrc_collab_aitq/corpus_renamed/', '')
    data_14M.append((f,e,loc_clean.split('/')[0],loc_clean.split('/')[1],idx))

counts_14M = dict()
for f, e, folder, domain, idx in data_14M:
    if domain in counts_14M:
        #print('yes')
        counts_14M[domain] += 1
    else:
        counts_14M[domain] = 1


with open('/fs/hestia_Hnrc/ict/bha101/nrc_collab_aitq/bt/domain_distribution/BT_Domain_distribution.en-fr', 'rb') as handle:
    counts = pickle.load(handle)

In [0]:
# missing Domains :
set(counts.keys()) - set(counts_14M.keys())

#Common domains :
counts.keys() & counts_14M.keys()

# Evaluate q_k and p_k values for KL :

qk=[]
pk=[]
for key_14 in counts_14M.keys():
    if key_14 in counts:
        qk.append(counts[key_14])
        pk.append(counts_14M[key_14])

import numpy as np

q_k = list(map((lambda x: x / np.sum(qk)), qk))
p_k = list(map((lambda x: x / np.sum(pk)), pk))

from scipy import stats
print(stats.entropy(p_k , q_k))
#0.011069468780077724

#KL :
q_k = np.array(q_k)
p_k = np.array(p_k)

KL_model = (p_k * np.log(p_k/q_k)).sum()
print("KL for the two distribution : " , KL_model)
